In [ ]:
import requests
import spacy

In [ ]:
ner = spacy.load("en_ner_craft_md")

In [ ]:
ner.pipe_names

In [ ]:
doc_ner = ner("Alterations in the hypocretin receptor 2 and preprohypocretin genes produce narcolepsy in some animals.")

for ent in doc_ner.ents:
    print(ent.text, ent.label_, ent.start_char, ent.end_char, ent.start, ent.end)

In [ ]:
doc_ner.to_json()

In [ ]:
ner.meta

In [ ]:
r = requests.get("http://127.0.0.1:8000/documents/?skip=0&limit=100")
j = r.json()

In [ ]:
ner_keys = ["title", "summary"]

In [ ]:
for doc in j:
    print(doc["id"])
    for key in ner_keys:
        print(key)
        doc_ner = ner(doc[key])

        for ent in doc_ner.ents:
            print("\t", ent.text, ent.label_, ent.start_char, ent.end_char, ent.start, ent.end)
    print("\n")

In [ ]:
post_json = {
  "id": "TEST:" + ent.text,
  "preferred_name": "TEST:" + ent.text,
  "entity_type": ent.label_,
  "synonyms": [
    ent.text
  ],
  "source": "CRAFT"
}

In [ ]:
r = requests.post("http://127.0.0.1:8000/entities/", json=post_json)

In [ ]:
r

In [ ]:
post_json = {
  "document_id": doc["id"],
  "entity_id": "TEST:" + ent.text,
  "text": ent.text,
  "document_section": key,
  "start_char": ent.start_char,
  "end_char": ent.end_char,
  "start_token": ent.start,
  "end_token": ent.end
}

In [ ]:
r = requests.post("http://127.0.0.1:8000/entity_mentions/", json=post_json)

In [ ]:
r